# Homework 2 - Higher Order Functions (10 pts)

In this homework, we will practice Python's higher order functions. Please note that you may only use higher order functions **without access to global variables**. Your expression should contain only **map()**, **filter()**, **sorted**, **reduce()** and your custom functions.

You are required to turn in this notebook with all the parts filled in between <>. Your notebook must be named BDM\_HW2\_HOF_NetID.ipynb.

We will be using the provided the Graduation Outcomes data set for cohorts from 2001 through 2006 (Classes of 2005 through 2010) from the NYC Department of Education (the data handle on NYC Open Data is **avir-tzek**). The data is available on online as **nyc_grads.csv**.

#### The first ten records of the data are shown below. Please note, the data are sorted by "Demographics", then "Borough", then "Cohort".

In [1]:
DO NOT RUN THIS CELL

,Demographic,Borough,Cohort,Total Cohort,Grads,Regents,Advanced Regents,Local,Dropped Out
0,Borough Total,Bronx,2001,11453,4913,2644,998,2271,2438
1,Borough Total,Bronx,2002,12032,5328,3118,992,2217,2140
2,Borough Total,Bronx,2003,13632,6389,3861,1255,2528,2472
3,Borough Total,Bronx,2004,14364,7448,4625,1395,2823,2303
4,Borough Total,Bronx,2005,15175,8229,5618,1544,2611,2147
5,Borough Total,Bronx,2006,15579,8524,6312,1558,2212,2402
6,Borough Total,Brooklyn,2001,19961,9758,6177,2829,3591,3547
7,Borough Total,Brooklyn,2002,20808,10337,7050,2865,3298,3369
8,Borough Total,Brooklyn,2003,21334,11064,7711,3239,3353,3198
9,Borough Total,Brooklyn,2004,22353,12303,8872,3741,3431,2973


In [1]:
import csv
from functools import reduce

## Task 1 (3 points)

In this data set, we would like to inspect whether there is a correlation between the dropped out percentage and the percentage of students graduated with advanced regents for schools in NYC. Thus, we must compute the ratio of **Dropped Out** and **Advanced Regents** for each borough, and for each year by dividing them by the **Total Cohort** value. Thus, the only useful records for us are those with the **Demographic** value equals to **'Borough Total'**, aka. you must filter the data to this type of demographic first.

Please complete the HOF expression below to transform each input record into a tuple of only 4 elements: **Borough**, **Cohort**, **Ratio_Advanced**, **Ratio_Dropped**. Note that, your expression must user **reader** as input. Though you are encouraged to write your helper functions (instead of lambdas) to make your expression concise, global variables are absolutely not allowed. No exception. The output must be exactly as below.

In [2]:
def borough_map(row):
    '''
    pick out necassary columns and calcuate ratios to total cohort
    '''
    if row['Demographic']=='Borough Total':
        return (row['Borough'], row['Cohort'], \
        float(row['Advanced Regents'])/float(row['Total Cohort']), \
        float(row['Dropped Out'])/float(row['Total Cohort']))
            
def b_filter(values):
    '''
    filter by only rows with borough total
    '''
    return filter(lambda x: x != None, values)

with open('nyc_grads.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    #output1 = reduce(b_reduce, map(borough_map, reader), []).items()
    output1 = list(b_filter(map(borough_map, reader)))
    
    
output1

[('Bronx', '2001', 0.08713874094123811, 0.21286999039552956),
 ('Bronx', '2002', 0.08244680851063829, 0.1778590425531915),
 ('Bronx', '2003', 0.09206279342723005, 0.1813380281690141),
 ('Bronx', '2004', 0.09711779448621553, 0.16033138401559455),
 ('Bronx', '2005', 0.10174629324546952, 0.1414827018121911),
 ('Bronx', '2006', 0.10000641889723345, 0.1541819115475961),
 ('Brooklyn', '2001', 0.14172636641450828, 0.1776965081909724),
 ('Brooklyn', '2002', 0.1376874279123414, 0.16190888119953864),
 ('Brooklyn', '2003', 0.15182338051935876, 0.14990156557607576),
 ('Brooklyn', '2004', 0.1673600858945108, 0.13300228157294322),
 ('Brooklyn', '2005', 0.16201692714164168, 0.11544489722806861),
 ('Brooklyn', '2006', 0.1676060783694819, 0.12314560129864274),
 ('Manhattan', '2001', 0.14609313338595106, 0.1548539857932123),
 ('Manhattan', '2002', 0.13904776052885687, 0.1294659436975414),
 ('Manhattan', '2003', 0.18207363642913754, 0.1245766986094099),
 ('Manhattan', '2004', 0.18582666754809282, 0.12176


## Task 2 (3 points)

Now given *output1* from Task 1, we need to compute the average ratio of dropped out and advanced regents per borough (i.e. averaging the numbers from 2001 to 2006 for each borough). Please complete the HOF expression below. Your output should be similar to the one provided. Your input is **output1**.

Note: If you could not finish Task 1, you could use the output information above as input to your Task 2. Please try to use the least memory possible for this task.

In [3]:
def reducer_sum(boroughs, value):
    '''
    find count of borough records and sum of ratios
    '''
    (borough, cohort, ratio_advanced, ratio_dropped) = value
    if boroughs.get(borough, -1) == -1:
        boroughs[borough] = {}
    boroughs[borough]['ratio_advanced'] = \
        boroughs[borough].get('ratio_advanced', 0) + ratio_advanced
    boroughs[borough]['ratio_dropped'] = \
        boroughs[borough].get('ratio_dropped', 0) + ratio_dropped
    boroughs[borough]['counts'] = boroughs[borough].get('counts', 0) + 1
    return boroughs

def mapp_avg(value):
    '''
    calculate average ratio for each borough 
    '''
    (borough, values) = value
    values['ratio_advanced'] = \
        values.get('ratio_advanced', 0)/ values.get('counts')
    values['ratio_dropped'] = \
        values.get('ratio_dropped', 0)/ values.get('counts')
    return (borough, values['ratio_advanced'], values['ratio_dropped'])

output2 = sorted(list(map(mapp_avg, reduce(reducer_sum, output1, {}).items())))

output2

[('Bronx', 0.0934198082513375, 0.17134384308218617),
 ('Brooklyn', 0.15470337770864048, 0.14351662251104022),
 ('Manhattan', 0.16519452307558916, 0.1223416853729485),
 ('Queens', 0.1769903541049419, 0.13903528573260707),
 ('Staten Island', 0.23144827521877342, 0.09375060031471814)]


## Task 3 (4 points)

In this task, you are asked to complete Homework 1 using only map(), filter(), reduce(), and/or sorted() higher-order functions. Note that, instead of writing to a CSV file, the output can be printed in the notebook, similar to what presented below (some differences in the output format are tolerable).

In [5]:
def sale_mapper(row):
    '''
    current row important information
    '''
    return ((row['Product ID'], row['Customer ID']), float(row['Item Cost']))

def reduce_cost(cost, values): 
    '''
    find total amount spent for each unique customer id and product pair
    a way to also reduce to unqiue customers per product
    '''
    (pair, i_cost) = values
    cost[pair] = cost.get(pair, 0) + i_cost
    return cost

def reduce_cust(sales, value):
    '''
    find total revenue and customers per product
    '''
    ((product, customer), cost) = value
    if sales.get(product, -1) == -1:
        sales[product] = {}
    sales[product]['cust'] = sales[product].get('cust', 0) + 1
    sales[product]['rev'] = sales[product].get('rev', 0) + cost
    return sales
    
def map_values(value):
    '''
    round the final revenue to 2 decimal places
    and format to string
    '''
    (product, values) = value
    return str(product)+ ' ' + str(values['cust'])+ ' ' + str(round(values['rev'], 2))

with open('sale.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output3 = '\n'.join(sorted(list(map(map_values, \
                              reduce(reduce_cust, \
                                     reduce(reduce_cost, \
                                            map(sale_mapper, reader), {}
                                           ).items(), {}
                                    ).items()
                             )
                         )
                    )
                       )

print (output3)

P02291 16 1181.97
P19498 17 989.99
P32565 17 1006.09
P33162 18 1210.92
P39328 17 1129.01
P58225 17 1349.82
P61235 18 959.02
P76615 18 1087.96
P82222 17 950.05
P92449 14 966.17
